In [1]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append("/home/td/Documents/sports_predictor")
sys.path.append("/home/td/Documents/sports_predictor/nba")

from process_data import DataManager

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# dm = DataManager(testing = 2000)
# dm.update_raw_datasets()
# dm.build_timeseries(4, False)
# dm.combine_timeseries(4, False)
# x1, x2, y, cols = dm.get_labeled_data(4, False)
# print(x1.shape, x2.shape, y.shape, len(cols))

In [3]:
# np.isnan(x1).astype(int).sum()

In [4]:
# x2_df = pd.DataFrame(data=x2,
#                     columns = cols)
# x2_df.isna().sum().sum()

In [5]:
# x2_df.head(10)

In [6]:
# team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
# player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)

# team_data.shape, player_data.shape

In [7]:
# team_data.head()

In [8]:
# player_data.head()

- Negatively correlated with win
- High stat def rating is highly correlated with losing (.51). Teams under pressure don't win that game.
- Personal fouls also correlated with losing (.13)
- Turnovers - (investigate, not intuitive)
- 

In [9]:
# team_data_corr = team_data.corr()
# team_data_corr = team_data_corr[(team_data_corr['win'] > .2)|(team_data_corr['win'] < -.08)]
# team_data_corr.sort_values('win')[['win']]

In [10]:
# player_data_corr = player_data.corr()
# player_data_corr = player_data_corr[(player_data_corr['win'] > .05)|(player_data_corr['win'] < -.05)]
# player_data_corr.sort_values('win')[['win']]

In [11]:

# past_n_game_obj = None
# with open('{data_path}/{db_name}.pkl'.format(data_path=data_path,
#                                                              db_name='past_n_game_dataset_32_False'), 'rb') as f:
#     past_n_game_dataset = pickle.load(f)
# past_n_game_dataset.keys()


In [12]:
# team_data[(team_data['team_tag'] == 'pho')&(team_data['month'] == 12)&(team_data['year'] == 2017)][['month', 'day', 'ast']].head(20)

In [13]:
# initial_team_data_columns = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
#                                           'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
#                                           'ft_pct',
#                                           'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
#                                           'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
#                                           'ts_pct',
#                                           'usg_pct', 'home', 'r1', 'r2', 'r3', 'r4']

# pd.DataFrame(data=past_n_game_dataset['pho']["['2017-12-31', 'phi', 'pho']"])


In [21]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results['use_resnet'] = results['use_resnet'].astype(int)
results['use_convolution_cell'] = results['use_convolution_cell'].astype(int)
results['recurrent_bi_direction'] = results['recurrent_bi_direction'].astype(int)
results['use_recurrent_cell'] = results['use_recurrent_cell'].astype(int)
results['use_recurrent_convolutional_cell'] = results['use_recurrent_convolutional_cell'].astype(int)
results['use_convolutional_recurrent_cell'] = results['use_convolutional_recurrent_cell'].astype(int)
results['use_dense_cell'] = results['use_dense_cell'].astype(int)
results['use_x1'] = results['use_x1'].astype(int)
results['use_x2'] = results['use_x2'].astype(int)

results.shape

(17, 33)

In [22]:
results.head()

,convolutional_filters,convolutional_kernel_size,convolutional_pool_size,dense_top_layers,dense_layers_width,recurrent_layers_width,pooling_algorithm,use_resnet,convolution_type,use_convolution_cell,convolutional_layers,recurrent_type,recurrent_bi_direction,use_recurrent_cell,recurrent_layers,use_recurrent_convolutional_cell,recurrent_convolutional_layers,use_convolutional_recurrent_cell,convolutional_recurrent_layers,use_dense_cell,dense_layers,use_x1,use_x2,optimizer_algorithm,activation,conv_layers_per_pooling_layer,input_shape1,input_shape2,num_of_params,history_lengths,transpose_history_data,model_training_time,accuracy
0,79,3,1,0,73,99,NaN,1,layers.Conv1D,0,2,layers.LSTM,0,0,2,0,2,1,1,1,2,1,0,optimizers.Adagrad(),elu,2,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN
1,16,3,1,2,37,27,layers.MaxPooling1D,1,layers.LocallyConnected1D,0,1,layers.GRU,0,0,1,0,2,1,2,1,1,1,0,optimizers.Adadelta(),relu,2,"(64, 306)","(177,)",NaN,NaN,NaN,NaN,NaN
2,20,1,4,1,87,119,layers.MaxPooling1D,0,layers.SeparableConv1D,1,1,layers.LSTM,0,0,2,0,1,1,1,1,2,1,1,optimizers.SGD(),relu,2,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN
3,84,7,3,0,48,86,layers.AveragePooling1D,1,layers.LocallyConnected1D,0,2,layers.RNN,0,1,2,1,1,1,1,0,2,1,1,optimizers.Nadam(),sigmoid,1,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN
4,106,4,1,0,40,95,layers.AveragePooling1D,0,layers.Conv1D,1,1,layers.RNN,0,0,1,1,2,1,1,1,2,1,1,optimizers.Adamax(),elu,2,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN


In [23]:
results.sort_values('accuracy', ascending = False).head(20)


,convolutional_filters,convolutional_kernel_size,convolutional_pool_size,dense_top_layers,dense_layers_width,recurrent_layers_width,pooling_algorithm,use_resnet,convolution_type,use_convolution_cell,convolutional_layers,recurrent_type,recurrent_bi_direction,use_recurrent_cell,recurrent_layers,use_recurrent_convolutional_cell,recurrent_convolutional_layers,use_convolutional_recurrent_cell,convolutional_recurrent_layers,use_dense_cell,dense_layers,use_x1,use_x2,optimizer_algorithm,activation,conv_layers_per_pooling_layer,input_shape1,input_shape2,num_of_params,history_lengths,transpose_history_data,model_training_time,accuracy
9,61,9,4,3,28,102,layers.MaxPooling1D,1,layers.Conv1D,0,2,layers.GRU,1,0,2,0,2,1,2,1,1,0,1,optimizers.Adadelta(),sigmoid,2,"(64, 306)","(177,)",6666.0,64.0,False,7.805381,0.664566
16,48,3,2,2,18,89,NaN,0,layers.Conv1D,0,1,layers.GRU,1,0,2,1,2,1,1,1,2,1,0,optimizers.SGD(),sigmoid,2,"(64, 306)","(177,)",862958.0,64.0,False,441.393796,0.650141
5,108,5,1,3,44,109,layers.MaxPooling1D,0,layers.Conv1D,1,1,layers.GRU,1,0,2,0,2,0,2,1,1,1,0,optimizers.Adamax(),relu,1,"(306, 64)","(177,)",1729466.0,64.0,True,28.522596,0.644669
10,80,6,3,3,16,107,layers.MaxPooling1D,1,layers.LocallyConnected1D,1,1,layers.GRU,1,0,1,1,1,1,1,0,2,0,1,optimizers.Adamax(),sigmoid,1,"(64, 306)","(177,)",3426.0,64.0,False,4.929801,0.634886
11,122,4,3,1,113,72,layers.MaxPooling1D,1,layers.SeparableConv1D,0,2,layers.LSTM,0,1,2,0,2,1,1,0,1,0,1,optimizers.SGD(),sigmoid,1,"(306, 64)","(177,)",20342.0,64.0,True,2.272222,0.634886
6,75,8,4,0,93,114,NaN,1,layers.SeparableConv1D,1,1,layers.LSTM,0,1,2,0,2,1,1,0,2,0,1,optimizers.SGD(),relu,2,"(306, 64)","(177,)",356.0,64.0,True,4.450621,0.357486
0,79,3,1,0,73,99,NaN,1,layers.Conv1D,0,2,layers.LSTM,0,0,2,0,2,1,1,1,2,1,0,optimizers.Adagrad(),elu,2,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN
1,16,3,1,2,37,27,layers.MaxPooling1D,1,layers.LocallyConnected1D,0,1,layers.GRU,0,0,1,0,2,1,2,1,1,1,0,optimizers.Adadelta(),relu,2,"(64, 306)","(177,)",NaN,NaN,NaN,NaN,NaN
2,20,1,4,1,87,119,layers.MaxPooling1D,0,layers.SeparableConv1D,1,1,layers.LSTM,0,0,2,0,1,1,1,1,2,1,1,optimizers.SGD(),relu,2,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN
3,84,7,3,0,48,86,layers.AveragePooling1D,1,layers.LocallyConnected1D,0,2,layers.RNN,0,1,2,1,1,1,1,0,2,1,1,optimizers.Nadam(),sigmoid,1,"(306, 64)","(177,)",NaN,NaN,NaN,NaN,NaN


In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results.sort_values('accuracy', ascending = False).head(10)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 4].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 8].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 16].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 32].sort_values('accuracy', ascending = False).head(20)